In [10]:
import json
import pandas as pd
import numpy as np
from datetime import datetime as dt

from _data.database import database

In [11]:
sec = json.load(open('_data/data_specs.json'))

In [12]:
db = database(sec, 1).data

Retrieving data from Yahoo Finance...
[*********************100%***********************]  74 of 74 completed
Retrieving data from BLS - Bureau of Labour Statistics...
Retrieving data from FRED - FED St Louis Economic Data...
Retrieving data from Census...
Retrieving data from BEA - Bureau of Economic Analyses...
Retrieving data from FED...
Retrieving data from Philly Fed...
Retrieving data from ONS - Office for National Statistics...


/Users/gcedism/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/gcedism/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/gcedism/opt/anaconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Retrieving data from Canada Central Bank...
Retrieving data from Eurostat Data...
Retrieving data from BCB...
Retrieving data from Manual csv File...


In [13]:
db.to_csv('_data/monthly_database.csv')

In [3]:
db = pd.read_csv('_data/monthly_database.csv', index_col = 0)
db.index = db.index.map(lambda x : dt.strptime(x, '%Y-%m-%d').date())

In [73]:
correls = []
for i in range(db.shape[1]) :
    for j in range(i, db.shape[1]):
        x = (db.iloc[:, i]
               .replace(',' , '' ,  regex=True)
               .replace(' - ' , np.nan ,  regex=True)
               .astype(float)
               .interpolate(limit_area = 'inside')
               .dropna())
        y = (db.iloc[:, j]
               .replace(',' , '' ,  regex=True)
               .replace(' - ' , np.nan ,  regex=True)
               .astype(float)
               .interpolate(limit_area = 'inside')
               .dropna())
        start = max(x.index[0], y.index[0])
        end = min(x.index[-1], y.index[-1])
        x = x[start:end]
        y = y[start:end]
        correls.append({
            'var1' : db.columns[i],
            'var2' : db.columns[j],
            'correl' : np.corrcoef(x, y)[0, 1]
        })

In [78]:
W = pd.DataFrame(correls)

In [90]:
W.loc[W['correl'] < 0.999].sort_values(by = 'correl')[:10]

,var1,var2,correl
29299,us_adp,uk_unemployment,-0.970855
3189,DX-Y.NYB,EURUSD=X,-0.966707
7114,IGOV,OJ=F,-0.959654
2263,CORP,OJ=F,-0.959500
10103,LQD,OJ=F,-0.958549
10252,LQD,eu_ip,-0.953842
11131,OJ=F,SPTI,-0.950734
5318,GOVT,OJ=F,-0.950100
11248,OJ=F,bra_reserves,-0.948137
15701,us_30y_rate,us_cpi_medicare,-0.947194


In [89]:
W.loc[W['correl'] < 0.999].sort_values(by = 'correl', ascending = False)[:10]

,var1,var2,correl
2258,CORP,LQD,0.998969
26476,uk_retail_sales_sa,uk_retail_sales_exfuel_sa,0.998787
16921,us_cpi_shelter,us_cpi_rent_sa,0.998660
15909,us_cpi,can_cpi,0.998132
16276,us_cpi_food,us_pce,0.998083
21631,can_cpi,us_pce,0.997960
17121,us_cpi_rent_sa,nor_cpi,0.997875
16055,us_cpi_core,us_cpi_shelter,0.997781
16086,us_cpi_core,nor_cpi,0.997503
23822,us_pce,eu_hicp,0.997496


In [92]:
db[['LQD', 'eu_ip']].dropna()

,LQD,eu_ip
2002-07-31,43.273296,160.3
2002-08-30,44.847366,142.4
2002-09-30,45.785275,159.6
2002-10-31,45.505592,159.1
2002-11-29,45.831390,154.6
...,...,...
2022-02-28,121.954109,71.2
2022-03-31,118.425049,82.4
2022-04-29,110.491447,79.8
2022-05-31,112.550896,79.8
